In [1]:
from creating_arrays import CHART_TIME_ARRAY,MONTH_ARRAY,SYMBOL_ARRAY
from constants import BINANCE_URL
import os
import re
import shutil
from selenium import webdriver

BINANCE_URL = BINANCE_URL.replace("?prefix=","")

https://data.binance.vision/?prefix=data/futures/um/monthly/klines/1000BTTCUSDT/
https://data.binance.vision/?prefix=data/futures/um/monthly/klines/1000LUNCBUSD/
https://data.binance.vision/?prefix=data/futures/um/monthly/klines/1000LUNCUSDT/
https://data.binance.vision/?prefix=data/futures/um/monthly/klines/1000SHIBBUSD/
https://data.binance.vision/?prefix=data/futures/um/monthly/klines/1000SHIBUSDT/
https://data.binance.vision/?prefix=data/futures/um/monthly/klines/1000XECUSDT/
https://data.binance.vision/?prefix=data/futures/um/monthly/klines/1INCHUSDT/
https://data.binance.vision/?prefix=data/futures/um/monthly/klines/AAVEUSDT/
https://data.binance.vision/?prefix=data/futures/um/monthly/klines/ADABUSD/
https://data.binance.vision/?prefix=data/futures/um/monthly/klines/ADAUSDT/
https://data.binance.vision/?prefix=data/futures/um/monthly/klines/AKROUSDT/
https://data.binance.vision/?prefix=data/futures/um/monthly/klines/ALGOUSDT/
https://data.binance.vision/?prefix=data/futures/um/mo

In [2]:
import datetime

# Get the current date
now = datetime.datetime.now()

# Format the date as a string in the desired format
date_string = now.strftime("%Y-%m")

# Print the date string
print(date_string)

idx = MONTH_ARRAY.index(date_string)
MONTH_ARRAY = MONTH_ARRAY[:idx+1]


2023-01


In [3]:
print(CHART_TIME_ARRAY)

['12h', '15m', '1d', '1h', '1m', '1mo', '1w', '2h', '30m', '3d', '3m', '4h', '5m', '6h', '8h']


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import glob

# Set up the ChromeOptions instance
options = Options()

# Set the download directory in the prefs dictionary
prefs = {"download.default_directory": "D:\\KISHORE\\Binance Data Perpetual Futures\\downloaded_data"}
options.add_experimental_option("prefs", prefs)

# Set up the webdriver
driver = webdriver.Chrome(options=options)

# Open the webpage
driver.get(BINANCE_URL)

# Set up the arrays of symbols, chart times, and months
symbols = [SYMBOL_ARRAY[44]]
chart_times = CHART_TIME_ARRAY
# chart_times = [CHART_TIME_ARRAY[12]]
months = MONTH_ARRAY

# Iterate over the symbols, chart times, and months
for symbol in symbols:
    for chart_time in chart_times:
        for month in months:
            # Construct the link
            link = f"{BINANCE_URL}{symbol}/{chart_time}/{symbol}-{chart_time}-{month}.zip"
#             print(link)
            # Open the link in a new tab
            driver.execute_script(f"window.open('{link}', '_blank');")

# # Wait for the downloads to finish
# driver.implicitly_wait(60)

# Close the webdriver
driver.close()


NoSuchWindowException: Message: no such window: No target with given id found
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x00C6F243]
	(No symbol) [0x00BF7FD1]
	(No symbol) [0x00AED04D]
	(No symbol) [0x00AE01BF]
	(No symbol) [0x00ADEA07]
	(No symbol) [0x00ADEEF7]
	(No symbol) [0x00ADEE88]
	(No symbol) [0x00AD2826]
	(No symbol) [0x00AD1F70]
	(No symbol) [0x00AD238C]
	(No symbol) [0x00B417A5]
	(No symbol) [0x00B38553]
	(No symbol) [0x00B1163C]
	(No symbol) [0x00B1269D]
	GetHandleVerifier [0x00F09A22+2655074]
	GetHandleVerifier [0x00EFCA24+2601828]
	GetHandleVerifier [0x00D18C0A+619850]
	GetHandleVerifier [0x00D17830+614768]
	(No symbol) [0x00C005FC]
	(No symbol) [0x00C05968]
	(No symbol) [0x00C05A55]
	(No symbol) [0x00C1051B]
	BaseThreadInitThunk [0x764400F9+25]
	RtlGetAppContainerNamedObjectPath [0x77607BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77607B8E+238]


In [5]:
import os
import re
import zipfile

import pandas as pd

# Set up the download directory
download_dir = "D:\\KISHORE\\Binance Data Perpetual Futures\\downloaded_data"

# Set up the output directory
output_dir = "D:\\KISHORE\\Binance Data Perpetual Futures\\extracted_data"

# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Set up an empty list for the data frames
        df_list = []

        # Compile the regular expression pattern
        pattern = re.compile(f"^{symbol}-{chart_time}-\d{{4}}-\d{{2}}\.zip$")

        # Iterate over the files in the download directory
        for file in os.listdir(download_dir):
            # Check if the file matches the pattern
            if pattern.match(file):
                # Construct the file path
                file_path = os.path.join(download_dir, file)

                # Extract the ZIP file
                with zipfile.ZipFile(file_path, "r") as zip_ref:
                    zip_ref.extractall(output_dir)

                # Construct the CSV file path
                csv_file_path = os.path.join(output_dir, f"{symbol}-{chart_time}{file[-12:-4]}.csv")

                # Read the CSV file into a data frame, ignoring the headers
                df = pd.read_csv(csv_file_path, header=None)

                # Reset the index of the dataframe and add it to the list
                df_list.append(df.reset_index(drop=True))

        # Concatenate the data frames in the list
        df_final = pd.concat(df_list, ignore_index=True)

        # Read the headers from the first CSV file
        headers = pd.read_csv(csv_file_path, nrows=1).columns

        # Set the headers as the column names of the final dataframe
        df_final.columns = headers

        # Set the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = os.path.join(output_dir, file_name)

        # Write the data frame to the Excel file
        df_final.to_csv(file_path, index=False)

        # Compile the regular expression pattern for the CSV files
        csv_pattern = re.compile(f"^{symbol}-{chart_time}-\d{{4}}-\d{{2}}\.csv$")

        # Delete the monthly CSV files
        for file in os.listdir(output_dir):
            if csv_pattern.match(file):
                os.remove(os.path.join(output_dir, file))
        
        # Delete the monthly ZIP files
        for file in os.listdir(download_dir):
            if pattern.match(file):
                os.remove(os.path.join(download_dir, file))

#Delete Download Directory
os.rmdir(download_dir)
